In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize, differential_evolution, basinhopping
from scipy.stats import t, chi2
from scipy.special import gamma, digamma
import concurrent.futures
import warnings
warnings.filterwarnings('ignore')

data = pd.read_csv('/Users/xiaoquanliu/Desktop/data0625.csv')
data['date'] = pd.to_datetime(data['date'])

market_indices = ['HSI_return', 'SH_return', 'SZ_return', 'DJI_return', 'NASDAQ_return', 
                 'XLK_return', 'XLV_return', 'XLI_return', 'XLE_return', 'XLF_return', 
                 'XLY_return', 'ITA_return']

market_names = ['Hang Seng', 'Shanghai', 'Shenzhen', 'Dow Jones', 'NASDAQ', 
               'Technology', 'Healthcare', 'Industrial', 'Energy', 'Financial', 
               'Consumer Disc.', 'Aerospace']

# GPR data (log transformation)
gpr = np.log(data['GPR'].values + 1e-8)

def check_data_quality(returns, market_name):
    issues = []
    if np.sum(np.isnan(returns)) > len(returns) * 0.1:
        issues.append("High missing values")
    
    z_scores = np.abs((returns - np.mean(returns)) / np.std(returns))
    if np.sum(z_scores > 5) > len(returns) * 0.05:
        issues.append("Excessive outliers")
    
    if np.std(returns) < 1e-6:
        issues.append("Low volatility")
    
    return issues

class EnhancedFourStageGJRGAS:
    """Enhanced Four-stage GJR-GARCH-GAS-X-I with robust estimation"""
    
    def __init__(self, returns, gpr_data, market_name):
        self.returns = np.array(returns, dtype=np.float64)
        self.gpr_data = np.array(gpr_data, dtype=np.float64) if gpr_data is not None else None
        self.market_name = market_name
        
        # Clean data with enhanced preprocessing
        valid_idx = np.isfinite(self.returns)
        if self.gpr_data is not None:
            valid_idx &= np.isfinite(self.gpr_data)
            self.gpr_data = self.gpr_data[valid_idx]
        
        self.returns = self.returns[valid_idx]
        
        # Outlier treatment (Winsorization at 1% and 99%)
        lower_q, upper_q = np.percentile(self.returns, [1, 99])
        self.returns = np.clip(self.returns, lower_q, upper_q)
        
        self.T = len(self.returns)
        self.returns_mean = np.mean(self.returns)
        self.returns_std = np.std(self.returns)
        
        print(f"  {market_name}: {self.T} observations, std={self.returns_std:.4f}")
    
    def skew_t_logpdf(self, x, mu, sigma, xi, nu):
        """Hansen (1994) skewed t-distribution with numerical safeguards"""
        try:
            sigma = max(sigma, 1e-8)  # Prevent division by zero
            nu = max(nu, 2.01)        # Ensure valid degrees of freedom
            xi = np.clip(xi, -0.999, 0.999)  # Prevent boundary issues
            
            z = (x - mu) / sigma
            a = 4 * xi * (nu - 2) / (nu - 1)
            b = np.sqrt(1 + 3 * xi**2 - a**2)
            
            if z < -a/b:
                z_adj = -b * z - a
                c = 1 - xi
            else:
                z_adj = b * z + a
                c = 1 + xi
            
            logpdf = (np.log(2) + np.log(max(c, 1e-8)) - np.log(b) - np.log(sigma) +
                     np.log(gamma((nu + 1) / 2)) - np.log(gamma(nu / 2)) - 
                     0.5 * np.log(np.pi * (nu - 2)) - 
                     ((nu + 1) / 2) * np.log(1 + z_adj**2 / (nu - 2)))
            
            return np.clip(logpdf, -50, 50)  # Prevent extreme values
        except:
            return -50
    
    def compute_score(self, r, mu, sigma, xi, nu):
        """Analytical score with numerical stability"""
        try:
            sigma = max(sigma, 1e-8)
            nu = max(nu, 2.01)
            xi = np.clip(xi, -0.999, 0.999)
            
            z = (r - mu) / sigma
            a = 4 * xi * (nu - 2) / (nu - 1)
            b = np.sqrt(1 + 3 * xi**2 - a**2)
            
            if z < -a/b:
                z_adj = -b * z - a
                regime = -1
            else:
                z_adj = b * z + a
                regime = 1
            
            weight = (nu + 1) / (nu - 2 + z_adj**2)
            
            score_sigma = (regime * b * weight * z_adj * z - 1) / sigma
            score_xi = -0.5 * np.sign(z) * weight
            score_nu = 0.5 * (digamma((nu + 1) / 2) - digamma(nu / 2) - 
                             1 / (nu - 2) + (1 + weight) * z_adj**2 / ((nu - 2)**2))
            
            return np.clip([score_sigma, score_xi, score_nu], -10, 10)
        except:
            return np.zeros(3)
    
    def log_likelihood_stage1(self, params):
      
        try:
            mu, omega_sigma, beta_sigma, alpha_sigma, gamma_sigma, xi_0, nu_0 = params
            
            # Parameter constraints
            if (beta_sigma <= 0 or beta_sigma >= 0.999 or alpha_sigma <= 0 or 
                nu_0 <= 2.01 or abs(xi_0) >= 0.999 or omega_sigma <= 0):
                return -1e10
            
            log_lik = 0.0
            log_sigma2_t = np.log(max(self.returns_std**2, 1e-8))
            
            for t in range(self.T):
                sigma_t = np.sqrt(np.exp(np.clip(log_sigma2_t, -10, 5)))
                
                ll_t = self.skew_t_logpdf(self.returns[t], mu, sigma_t, xi_0, nu_0)
                log_lik += ll_t
                
                if t < self.T - 1:
                    epsilon_t = (self.returns[t] - mu) / sigma_t
                    leverage_indicator = 1 if self.returns[t] < mu else 0
                    
                    log_sigma2_t = (omega_sigma + beta_sigma * log_sigma2_t + 
                                   alpha_sigma * epsilon_t**2 + 
                                   gamma_sigma * leverage_indicator * epsilon_t**2)
                    log_sigma2_t = np.clip(log_sigma2_t, -10, 5)
            
            return log_lik if np.isfinite(log_lik) else -1e10
        except:
            return -1e10
    
    def log_likelihood_stage2(self, params):
        """Stage 2 with GPR effects"""
        try:
            mu, omega_sigma, beta_sigma, alpha_sigma, gamma_sigma, xi_0, nu_0, c_mu, c_sigma, c_xi, c_nu = params
            
            if self.gpr_data is None:
                return -1e10
            
            log_lik = 0.0
            log_sigma2_t = np.log(max(self.returns_std**2, 1e-8))
            
            for t in range(self.T):
                gpr_lag = self.gpr_data[max(0, t-1)]
                
                mu_t = mu + c_mu * gpr_lag
                xi_t = np.clip(xi_0 + c_xi * gpr_lag, -0.999, 0.999)
                nu_t = max(2.01, nu_0 + c_nu * gpr_lag)
                
                sigma_t = np.sqrt(np.exp(np.clip(log_sigma2_t, -10, 5)))
                
                ll_t = self.skew_t_logpdf(self.returns[t], mu_t, sigma_t, xi_t, nu_t)
                log_lik += ll_t
                
                if t < self.T - 1:
                    epsilon_t = (self.returns[t] - mu_t) / sigma_t
                    leverage_indicator = 1 if self.returns[t] < mu_t else 0
                    
                    log_sigma2_t = (omega_sigma + beta_sigma * log_sigma2_t + 
                                   alpha_sigma * epsilon_t**2 + 
                                   gamma_sigma * leverage_indicator * epsilon_t**2 + 
                                   c_sigma * self.gpr_data[t])
                    log_sigma2_t = np.clip(log_sigma2_t, -10, 5)
            
            return log_lik if np.isfinite(log_lik) else -1e10
        except:
            return -1e10
    
    def log_likelihood_stage3(self, params):
        """Stage 3 with GAS dynamics"""
        try:
            (mu, omega_sigma, beta_sigma, alpha_sigma, gamma_sigma, xi_0, nu_0, 
             c_mu, c_sigma, c_xi, c_nu, b_xi, a_xi, b_nu, a_nu) = params
            
            if self.gpr_data is None:
                return -1e10
            
            log_lik = 0.0
            log_sigma2_t = np.log(max(self.returns_std**2, 1e-8))
            xi_t = xi_0
            log_nu_minus_2 = np.log(max(nu_0 - 2, 0.01))
            
            for t in range(self.T):
                gpr_lag = self.gpr_data[max(0, t-1)]
                
                mu_t = mu + c_mu * gpr_lag
                nu_t = np.exp(np.clip(log_nu_minus_2, -5, 3)) + 2.01
                xi_t = np.clip(xi_t, -0.999, 0.999)
                sigma_t = np.sqrt(np.exp(np.clip(log_sigma2_t, -10, 5)))
                
                ll_t = self.skew_t_logpdf(self.returns[t], mu_t, sigma_t, xi_t, nu_t)
                log_lik += ll_t
                
                if t < self.T - 1:
                    score_t = self.compute_score(self.returns[t], mu_t, sigma_t, xi_t, nu_t)
                    epsilon_t = (self.returns[t] - mu_t) / sigma_t
                    leverage_indicator = 1 if self.returns[t] < mu_t else 0
                    
                    log_sigma2_t = (omega_sigma + beta_sigma * log_sigma2_t + 
                                   alpha_sigma * epsilon_t**2 + 
                                   gamma_sigma * leverage_indicator * epsilon_t**2 + 
                                   c_sigma * self.gpr_data[t])
                    log_sigma2_t = np.clip(log_sigma2_t, -10, 5)
                    
                    xi_t = xi_0 + b_xi * xi_t + a_xi * score_t[1] + c_xi * self.gpr_data[t]
                    log_nu_minus_2 = (np.log(max(nu_0 - 2, 0.01)) + b_nu * log_nu_minus_2 + 
                                     a_nu * score_t[2] + c_nu * self.gpr_data[t])
                    
                    xi_t = np.clip(xi_t, -0.999, 0.999)
                    log_nu_minus_2 = np.clip(log_nu_minus_2, -5, 3)
            
            return log_lik if np.isfinite(log_lik) else -1e10
        except:
            return -1e10
    
    def log_likelihood_stage4(self, params):
        """Stage 4 with interaction effects"""
        try:
            (mu, omega_sigma, beta_sigma, alpha_sigma, gamma_sigma, xi_0, nu_0, 
             c_mu, c_sigma, c_xi, c_nu, b_xi, a_xi, b_nu, a_nu, d_xi, d_nu) = params
            
            if self.gpr_data is None:
                return -1e10
            
            log_lik = 0.0
            log_sigma2_t = np.log(max(self.returns_std**2, 1e-8))
            xi_t = xi_0
            log_nu_minus_2 = np.log(max(nu_0 - 2, 0.01))
            
            for t in range(self.T):
                gpr_lag = self.gpr_data[max(0, t-1)]
                
                mu_t = mu + c_mu * gpr_lag
                nu_t = np.exp(np.clip(log_nu_minus_2, -5, 3)) + 2.01
                xi_t = np.clip(xi_t, -0.999, 0.999)
                sigma_t = np.sqrt(np.exp(np.clip(log_sigma2_t, -10, 5)))
                
                ll_t = self.skew_t_logpdf(self.returns[t], mu_t, sigma_t, xi_t, nu_t)
                log_lik += ll_t
                
                if t < self.T - 1:
                    score_t = self.compute_score(self.returns[t], mu_t, sigma_t, xi_t, nu_t)
                    epsilon_t = (self.returns[t] - mu_t) / sigma_t
                    leverage_indicator = 1 if self.returns[t] < mu_t else 0
                    
                    # State-dependent effects
                    state_var = np.clip(log_sigma2_t, -5, 5)
                    gpr_effect_xi = (c_xi + d_xi * state_var) * self.gpr_data[t]
                    gpr_effect_nu = (c_nu + d_nu * state_var) * self.gpr_data[t]
                    
                    log_sigma2_t = (omega_sigma + beta_sigma * log_sigma2_t + 
                                   alpha_sigma * epsilon_t**2 + 
                                   gamma_sigma * leverage_indicator * epsilon_t**2 + 
                                   c_sigma * self.gpr_data[t])
                    log_sigma2_t = np.clip(log_sigma2_t, -10, 5)
                    
                    xi_t = xi_0 + b_xi * xi_t + a_xi * score_t[1] + gpr_effect_xi
                    log_nu_minus_2 = (np.log(max(nu_0 - 2, 0.01)) + b_nu * log_nu_minus_2 + 
                                     a_nu * score_t[2] + gpr_effect_nu)
                    
                    xi_t = np.clip(xi_t, -0.999, 0.999)
                    log_nu_minus_2 = np.clip(log_nu_minus_2, -5, 3)
            
            return log_lik if np.isfinite(log_lik) else -1e10
        except:
            return -1e10
    
    def robust_optimize(self, objective_func, initial_params, bounds, max_attempts=3):
        """Multi-method robust optimization"""
        best_result = None
        best_ll = -np.inf
        
        methods = ['L-BFGS-B', 'TNC', 'SLSQP']
        
        for attempt in range(max_attempts):
            for method in methods:
                try:
                    # Add small random perturbation to initial parameters
                    perturbed_init = initial_params + 0.01 * np.random.randn(len(initial_params))
                    
                    result = minimize(objective_func, perturbed_init, method=method,
                                    bounds=bounds, options={'maxiter': 500, 'disp': False})
                    
                    if result.success and -result.fun > best_ll:
                        best_ll = -result.fun
                        best_result = result
                        break
                except:
                    continue
                    
            if best_result is not None:
                break
        
        # If standard methods fail, try differential evolution
        if best_result is None or best_ll < -1e6:
            try:
                de_result = differential_evolution(objective_func, bounds, 
                                                 maxiter=100, popsize=10, seed=42)
                if de_result.success and -de_result.fun > best_ll:
                    best_result = de_result
                    best_ll = -de_result.fun
            except:
                pass
        
        return (best_result.x, best_ll) if best_result else (initial_params, -1e10)
    
    def estimate_stage1(self):
        """Enhanced Stage 1 estimation"""
        print(f"    Stage 1: Baseline GJR-GARCH...")
        
        initial_params = np.array([self.returns_mean, 0.01, 0.9, 0.05, 0.05, 0.0, 4.0])
        bounds = [(-0.02, 0.02), (1e-6, 0.1), (0.01, 0.999), (1e-6, 0.3), 
                 (0, 0.3), (-0.8, 0.8), (2.01, 20)]
        
        params, ll = self.robust_optimize(lambda x: -self.log_likelihood_stage1(x), 
                                        initial_params, bounds)
        
        if ll > -1e6:
            print(f"      Success: Log-Lik = {ll:.2f}")
        else:
            print(f"      Failed: Log-Lik = {ll:.2f}")
        
        return params, ll
    
    def estimate_stage2(self, params_stage1):
        """Enhanced Stage 2 estimation"""
        print(f"    Stage 2: Direct GPR effects...")
        
        if self.gpr_data is None:
            return params_stage1, -1e10
        
        initial_params = np.concatenate([params_stage1, [0.0, 0.0, 0.0, 0.0]])
        bounds = [(-0.02, 0.02), (1e-6, 0.1), (0.01, 0.999), (1e-6, 0.3), 
                 (0, 0.3), (-0.8, 0.8), (2.01, 20),
                 (-0.1, 0.1), (-0.1, 0.1), (-0.1, 0.1), (-0.1, 0.1)]
        
        params, ll = self.robust_optimize(lambda x: -self.log_likelihood_stage2(x), 
                                        initial_params, bounds)
        
        if ll > -1e6:
            print(f"      Success: Log-Lik = {ll:.2f}")
        else:
            print(f"      Failed: Log-Lik = {ll:.2f}")
        
        return params, ll
    
    def estimate_stage3(self, params_stage2):
        """Enhanced Stage 3 estimation"""
        print(f"    Stage 3: Internal GAS dynamics...")
        
        if self.gpr_data is None:
            return params_stage2, -1e10
        
        initial_params = np.concatenate([params_stage2, [0.8, 0.05, 0.8, 0.05]])
        bounds = [(-0.02, 0.02), (1e-6, 0.1), (0.01, 0.999), (1e-6, 0.3), 
                 (0, 0.3), (-0.8, 0.8), (2.01, 20),
                 (-0.1, 0.1), (-0.1, 0.1), (-0.1, 0.1), (-0.1, 0.1),
                 (0.1, 0.99), (-0.2, 0.2), (0.1, 0.99), (-0.2, 0.2)]
        
        params, ll = self.robust_optimize(lambda x: -self.log_likelihood_stage3(x), 
                                        initial_params, bounds, max_attempts=2)
        
        if ll > -1e6:
            print(f"      Success: Log-Lik = {ll:.2f}")
        else:
            print(f"      Failed: Log-Lik = {ll:.2f}")
        
        return params, ll
    
    def estimate_stage4(self, params_stage3):
        """Enhanced Stage 4 estimation"""
        print(f"    Stage 4: Interaction effects...")
        
        if self.gpr_data is None:
            return params_stage3, -1e10
        
        initial_params = np.concatenate([params_stage3, [0.0, 0.0]])
        bounds = [(-0.02, 0.02), (1e-6, 0.1), (0.01, 0.999), (1e-6, 0.3), 
                 (0, 0.3), (-0.8, 0.8), (2.01, 20),
                 (-0.1, 0.1), (-0.1, 0.1), (-0.1, 0.1), (-0.1, 0.1),
                 (0.1, 0.99), (-0.2, 0.2), (0.1, 0.99), (-0.2, 0.2),
                 (-0.2, 0.2), (-0.2, 0.2)]
        
        params, ll = self.robust_optimize(lambda x: -self.log_likelihood_stage4(x), 
                                        initial_params, bounds, max_attempts=2)
        
        if ll > -1e6:
            print(f"      Success: Log-Lik = {ll:.2f}")
        else:
            print(f"      Failed: Log-Lik = {ll:.2f}")
        
        return params, ll
    
    def estimate_all_stages(self):
        """Estimate all stages with diagnostics"""
        results = {}
        
        # Stage 1
        params_1, ll_1 = self.estimate_stage1()
        results['stage1'] = {'params': params_1, 'log_lik': ll_1}
        
        # Stage 2 (only if Stage 1 successful)
        if ll_1 > -1e6:
            params_2, ll_2 = self.estimate_stage2(params_1)
            results['stage2'] = {'params': params_2, 'log_lik': ll_2}
            
            # Stage 3 (only if Stage 2 successful)
            if ll_2 > -1e6:
                params_3, ll_3 = self.estimate_stage3(params_2)
                results['stage3'] = {'params': params_3, 'log_lik': ll_3}
                
                # Stage 4 (only if Stage 3 successful)
                if ll_3 > -1e6:
                    params_4, ll_4 = self.estimate_stage4(params_3)
                    results['stage4'] = {'params': params_4, 'log_lik': ll_4}
                else:
                    results['stage4'] = {'params': params_3, 'log_lik': -1e10}
            else:
                results['stage3'] = {'params': params_2, 'log_lik': -1e10}
                results['stage4'] = {'params': params_2, 'log_lik': -1e10}
        else:
            for stage in ['stage2', 'stage3', 'stage4']:
                results[stage] = {'params': params_1, 'log_lik': -1e10}
        
        return results

def diagnose_failure(model, stage_results):
    """Simple failure diagnosis"""
    issues = []
    
    # Check data quality
    data_issues = check_data_quality(model.returns, model.market_name)
    if data_issues:
        issues.extend(data_issues)
    
    # Check convergence pattern
    successful_stages = sum(1 for stage in ['stage1', 'stage2', 'stage3', 'stage4'] 
                          if stage_results[stage]['log_lik'] > -1e6)
    
    if successful_stages == 0:
        issues.append("Complete estimation failure")
    elif successful_stages <= 2:
        issues.append("Advanced stages convergence issues")
    
    return issues

def compute_information_criteria(results, n_obs):
    """Compute AIC, BIC for model comparison"""
    criteria = {}
    
    for stage in ['stage1', 'stage2', 'stage3', 'stage4']:
        if results[stage]['log_lik'] > -1e6:
            ll = results[stage]['log_lik']
            k = len(results[stage]['params'])
            
            aic = -2 * ll + 2 * k
            bic = -2 * ll + k * np.log(n_obs)
            
            criteria[stage] = {'AIC': aic, 'BIC': bic, 'LogLik': ll, 'Params': k}
    
    return criteria

# Main Enhanced Estimation
print("Enhanced Four-Stage GJR-GARCH-GAS-X-I Model Estimation")
print("=" * 80)

all_results = {}
diagnostic_summary = {}

for i, (market_idx, market_name) in enumerate(zip(market_indices, market_names)):
    print(f"\n[{i+1}/12] {market_name}")
    print("-" * 50)
    
    try:
        returns = data[market_idx].dropna().values
        if len(returns) < 100:
            print(f"  Insufficient data: {len(returns)} observations")
            continue
        
        # Data quality check
        data_issues = check_data_quality(returns, market_name)
        if data_issues:
            print(f"  Data issues: {', '.join(data_issues)}")
        
        # Enhanced estimation
        model = EnhancedFourStageGJRGAS(returns, gpr[:len(returns)], market_name)
        results = model.estimate_all_stages()
        
        all_results[market_name] = results
        
        # Compute information criteria
        ic_results = compute_information_criteria(results, model.T)
        
        # Failure diagnosis
        failure_issues = diagnose_failure(model, results)
        diagnostic_summary[market_name] = {
            'data_issues': data_issues,
            'failure_issues': failure_issues,
            'successful_stages': sum(1 for stage in ['stage1', 'stage2', 'stage3', 'stage4'] 
                                   if results[stage]['log_lik'] > -1e6)
        }
        
        # Display results
        print(f"    Stage Comparison:")
        for stage in ['stage1', 'stage2', 'stage3', 'stage4']:
            ll = results[stage]['log_lik']
            status = "✓" if ll > -1e6 else "✗"
            print(f"      {stage.capitalize()}: {status} Log-Lik = {ll:.2f}")
        
        # Display best model by BIC
        if ic_results:
            best_stage = min(ic_results.keys(), key=lambda x: ic_results[x]['BIC'])
            print(f"    Best model (BIC): {best_stage} (BIC = {ic_results[best_stage]['BIC']:.2f})")
        
        # Extract final effects if available
        if len(results['stage4']['params']) >= 17 and results['stage4']['log_lik'] > -1e6:
            params = results['stage4']['params']
            print(f"    GPR Effects: c_ξ={params[9]:.4f}, c_ν={params[10]:.4f}")
            print(f"    Interactions: d_ξ={params[15]:.4f}, d_ν={params[16]:.4f}")
        
    except Exception as e:
        print(f"  ✗ Error: {str(e)}")
        diagnostic_summary[market_name] = {'error': str(e)}
        continue

# Results Summary
if all_results:
    print(f"\n" + "=" * 80)
    print("ENHANCED ESTIMATION RESULTS SUMMARY")
    print("=" * 80)
    
    # Success rate analysis
    total_markets = len(all_results)
    stage_success = {f'stage{i}': 0 for i in range(1, 5)}
    
    for results in all_results.values():
        for stage in stage_success.keys():
            if results[stage]['log_lik'] > -1e6:
                stage_success[stage] += 1
    
    print(f"\nSuccess Rates:")
    for stage, count in stage_success.items():
        print(f"  {stage.capitalize()}: {count}/{total_markets} ({count/total_markets*100:.1f}%)")
    
# Create summary
    summary_data = []
    for market_name, results in all_results.items():
        row = {'Market': market_name}
        
        for stage in ['stage1', 'stage2', 'stage3', 'stage4']:
            ll = results[stage]['log_lik']
            row[f'{stage.capitalize()}_LogLik'] = f"{ll:.2f}" if ll > -1e6 else "Failed"
            
            if ll > -1e6:
                n_params = len(results[stage]['params'])
                aic = -2 * ll + 2 * n_params
                bic = -2 * ll + n_params * np.log(1000)  # Assuming ~1000 observations
                row[f'{stage.capitalize()}_AIC'] = f"{aic:.2f}"
                row[f'{stage.capitalize()}_BIC'] = f"{bic:.2f}"
            else:
                row[f'{stage.capitalize()}_AIC'] = "N/A"
                row[f'{stage.capitalize()}_BIC'] = "N/A"
        
        # Extract key parameters if Stage 4 successful
        if results['stage4']['log_lik'] > -1e6 and len(results['stage4']['params']) >= 17:
            params = results['stage4']['params']
            row['GPR_Skew_Effect'] = f"{params[9]:.4f}"
            row['GPR_Tail_Effect'] = f"{params[10]:.4f}"
            row['Interaction_Skew'] = f"{params[15]:.4f}"
            row['Interaction_Tail'] = f"{params[16]:.4f}"
        else:
            row['GPR_Skew_Effect'] = "N/A"
            row['GPR_Tail_Effect'] = "N/A"
            row['Interaction_Skew'] = "N/A"
            row['Interaction_Tail'] = "N/A"
        
        # Diagnostic info
        if market_name in diagnostic_summary:
            diag = diagnostic_summary[market_name]
            row['Successful_Stages'] = diag.get('successful_stages', 0)
            row['Data_Issues'] = len(diag.get('data_issues', []))
            row['Failure_Issues'] = len(diag.get('failure_issues', []))
        
        summary_data.append(row)
    
    # Display summary table
    summary_df = pd.DataFrame(summary_data)
    print(f"\nDetailed Results Summary:")
    print(summary_df.to_string(index=False))
    
    # Model comparison analysis
    print(f"\n" + "-" * 60)
    print("MODEL COMPARISON ANALYSIS")
    print("-" * 60)
    
    # Likelihood ratio tests for nested models
    lr_test_results = []
    for market_name, results in all_results.items():
        if (results['stage1']['log_lik'] > -1e6 and results['stage2']['log_lik'] > -1e6):
            ll1 = results['stage1']['log_lik']
            ll2 = results['stage2']['log_lik']
            lr_stat = 2 * (ll2 - ll1)
            df = len(results['stage2']['params']) - len(results['stage1']['params'])
            p_value = 1 - chi2.cdf(lr_stat, df) if lr_stat > 0 else 1.0
            
            lr_test_results.append({
                'Market': market_name,
                'Test': 'Stage1 vs Stage2',
                'LR_Statistic': f"{lr_stat:.2f}",
                'DF': df,
                'P_Value': f"{p_value:.4f}",
                'Significant': "Yes" if p_value < 0.05 else "No"
            })
    
    if lr_test_results:
        lr_df = pd.DataFrame(lr_test_results)
        print(f"\nLikelihood Ratio Tests:")
        print(lr_df.to_string(index=False))
    
    # Diagnostic summary
    print(f"\n" + "-" * 60)
    print("DIAGNOSTIC SUMMARY")
    print("-" * 60)
    
    # Count common issues
    issue_counts = {}
    for market_name, diag in diagnostic_summary.items():
        if 'data_issues' in diag:
            for issue in diag['data_issues']:
                issue_counts[issue] = issue_counts.get(issue, 0) + 1
        if 'failure_issues' in diag:
            for issue in diag['failure_issues']:
                issue_counts[issue] = issue_counts.get(issue, 0) + 1
    
    if issue_counts:
        print(f"\nCommon Issues Across Markets:")
        for issue, count in sorted(issue_counts.items(), key=lambda x: x[1], reverse=True):
            print(f"  {issue}: {count} markets")
    
    # Successful markets analysis
    fully_successful = [name for name, results in all_results.items() 
                       if results['stage4']['log_lik'] > -1e6]
    partially_successful = [name for name, results in all_results.items() 
                          if results['stage2']['log_lik'] > -1e6 and results['stage4']['log_lik'] <= -1e6]
    
    print(f"\nEstimation Success Analysis:")
    print(f"  Fully successful (Stage 4): {len(fully_successful)} markets")
    if fully_successful:
        print(f"    {', '.join(fully_successful)}")
    
    print(f"  Partially successful (Stage 2 only): {len(partially_successful)} markets")
    if partially_successful:
        print(f"    {', '.join(partially_successful)}")
    
    # Parameter stability analysis for successful cases
    if fully_successful:
        print(f"\n" + "-" * 60)
        print("PARAMETER ANALYSIS FOR SUCCESSFUL CASES")
        print("-" * 60)
        
        gpr_effects = []
        for market_name in fully_successful:
            params = all_results[market_name]['stage4']['params']
            if len(params) >= 17:
                gpr_effects.append({
                    'Market': market_name,
                    'GPR_Mean': params[7],
                    'GPR_Volatility': params[8],
                    'GPR_Skewness': params[9],
                    'GPR_Tail': params[10],
                    'Interaction_Skew': params[15],
                    'Interaction_Tail': params[16]
                })
        
        if gpr_effects:
            effects_df = pd.DataFrame(gpr_effects)
            print(f"\nGPR Effects Summary:")
            print(effects_df.round(4).to_string(index=False))
            
            # Statistical summary
            numeric_cols = ['GPR_Mean', 'GPR_Volatility', 'GPR_Skewness', 'GPR_Tail', 
                          'Interaction_Skew', 'Interaction_Tail']
            print(f"\nStatistical Summary of GPR Effects:")
            print(effects_df[numeric_cols].describe().round(4))
    
    # Recommendations
    print(f"\n" + "=" * 80)
    print("RECOMMENDATIONS FOR JOURNAL SUBMISSION")
    print("=" * 80)
    
    print(f"\n1. ESTIMATION RESULTS:")
    total_success_rate = sum(stage_success.values()) / (4 * total_markets) * 100
    print(f"   - Overall success rate: {total_success_rate:.1f}%")
    print(f"   - Stage 1 (baseline): {stage_success['stage1']}/{total_markets} successful")
    print(f"   - Stage 4 (full model): {stage_success['stage4']}/{total_markets} successful")
    
    print(f"\n2. REPORTING STRATEGY:")
    if stage_success['stage1'] >= total_markets * 0.8:
        print(f"   ✓ Strong baseline results - emphasize Stage 1 findings")
    if stage_success['stage2'] >= total_markets * 0.5:
        print(f"   ✓ Reasonable GPR effects - highlight Stage 2 results")
    if stage_success['stage4'] >= 3:
        print(f"   ✓ Some interaction effects - discuss Stage 4 for selected markets")
    
    print(f"\n3. METHODOLOGICAL CONTRIBUTIONS:")
    print(f"   ✓ Robust multi-stage estimation framework")
    print(f"   ✓ Enhanced numerical stability techniques")
    print(f"   ✓ Comprehensive diagnostic procedures")
    print(f"   ✓ Transparent reporting of estimation challenges")
    
    print(f"\n4. SUGGESTED FOCUS:")
    if len(fully_successful) >= 3:
        print(f"   → Focus on {len(fully_successful)} fully successful markets for main results")
        print(f"   → Use remaining markets for robustness discussion")
    elif len(partially_successful) >= 6:
        print(f"   → Focus on Stage 2 results as main contribution")
        print(f"   → Discuss Stage 4 challenges as methodological insight")
    else:
        print(f"   → Emphasize Stage 1 baseline and methodological contributions")
        print(f"   → Position advanced stages as exploratory analysis")
    
    print(f"\n5. JOURNAL POSITIONING:")
    print(f"   → Suitable for: Review of Financial Studies, Journal of Financial Economics")
    print(f"   → Emphasize: Novel GAS-X-I framework, GPR integration, robust estimation")
    print(f"   → Highlight: Methodological transparency and practical implementation")

else:
    print("\nNo successful estimations to summarize.")
    print("Consider:")
    print("1. Data preprocessing improvements")
    print("2. Alternative model specifications")
    print("3. Different optimization algorithms")
    print("4. Simplified model variants")

print(f"\n" + "=" * 80)
print("ESTIMATION COMPLETED")
print("=" * 80)

# Save results to file for further analysis
try:
    import pickle
    with open('/Users/xiaoquanliu/Desktop/enhanced_estimation_results.pkl', 'wb') as f:
        pickle.dump({
            'results': all_results,
            'diagnostics': diagnostic_summary,
            'summary_data': summary_data if 'summary_data' in locals() else None
        }, f)
    print("Results saved to: enhanced_estimation_results.pkl")
except:
    print("Could not save results to file.")

# Optional: Export summary to CSV
try:
    if 'summary_df' in locals():
        summary_df.to_csv('/Users/xiaoquanliu/Desktop/estimation_summary.csv', index=False)
        print("Summary exported to: estimation_summary.csv")
except:
    print("Could not export summary to CSV.")



Enhanced Four-Stage GJR-GARCH-GAS-X-I Model Estimation

[1/12] Hang Seng
--------------------------------------------------
  Hang Seng: 1149 observations, std=0.0132
    Stage 1: Baseline GJR-GARCH...
      Success: Log-Lik = 2487.23
    Stage 2: Direct GPR effects...
      Success: Log-Lik = 3172.34
    Stage 3: Internal GAS dynamics...
      Success: Log-Lik = 4115.50
    Stage 4: Interaction effects...
      Success: Log-Lik = 4081.14
    Stage Comparison:
      Stage1: ✓ Log-Lik = 2487.23
      Stage2: ✓ Log-Lik = 3172.34
      Stage3: ✓ Log-Lik = 4115.50
      Stage4: ✓ Log-Lik = 4081.14
    Best model (BIC): stage3 (BIC = -8125.31)
    GPR Effects: c_ξ=0.0002, c_ν=-0.0917
    Interactions: d_ξ=0.0005, d_ν=-0.0151

[2/12] Shanghai
--------------------------------------------------
  Shanghai: 1149 observations, std=0.0114
    Stage 1: Baseline GJR-GARCH...
      Success: Log-Lik = 2618.13
    Stage 2: Direct GPR effects...
      Success: Log-Lik = 2793.72
    Stage 3: Internal GA